# Processing the data (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

A simple example of fine-tuning a pre-trained model using sample data.

In [ ]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# This is new
batch["labels"] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

 Let's use the **MRPC** dataset! This is one of the 10 datasets composing the **GLUE benchmark**, which is an academic benchmark that is used to measure the performance of ML models across 10 different text classification tasks.

 This command downloads and caches the dataset, by default in ~/.cache/huggingface/datasets. Customize your cache folder by setting the **HF_HOME** environment variable.

In [2]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

**Analyze dataset**
1. Access each element using indices.
2. Get details of each feature.

In [19]:
raw_train_dataset = raw_datasets["train"]
# Access each example using Indices
print(raw_train_dataset[0])
print(raw_train_dataset[15])
print(raw_datasets["validation"][86])

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', 'label': 1, 'idx': 0}
{'sentence1': 'Rudder was most recently senior vice president for the Developer & Platform Evangelism Business .', 'sentence2': 'Senior Vice President Eric Rudder , formerly head of the Developer and Platform Evangelism unit , will lead the new entity .', 'label': 0, 'idx': 16}
{'sentence1': 'He was arrested Friday night at an Alpharetta seafood restaurant while dining with his wife , singer Whitney Houston .', 'sentence2': 'He was arrested again Friday night at an Alpharetta restaurant where he was having dinner with his wife .', 'label': 1, 'idx': 796}


In [4]:
# Get information of each feature
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [17]:
#label to id mapping and vice-versa
print(raw_train_dataset.features['label'].num_classes)
print(raw_train_dataset.features['label'].names)
print(raw_train_dataset.features['label'].int2str(0))
print(raw_train_dataset.features['label'].int2str(1))

2
['not_equivalent', 'equivalent']
not_equivalent
equivalent


## Preprocessing Dataset

The model **bert-base-uncased** was trained with two objectives:- Masked language modeling (MLM) and Next sentence prediction (NSP).

We will look at the preprocessing for NSP.
**Next sentence prediction (NSP)**: the models **concatenates two masked sentences** as inputs during pretraining. Sometimes they correspond to sentences that were next to each other in the original text, sometimes not. The model then has to predict if the two sentences were following each other or not.

In [20]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

We pass the two sentences together. Here, the tokenizer returns **token_type_ids** which tells the model which part of the input is the first sentence and which is the second sentence

In [24]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [27]:
print(tokenizer.convert_ids_to_tokens(inputs["input_ids"]))
print(inputs['token_type_ids'])
print(list(zip(tokenizer.convert_ids_to_tokens(inputs["input_ids"]),inputs['token_type_ids'])))

['[CLS]', 'this', 'is', 'the', 'first', 'sentence', '.', '[SEP]', 'this', 'is', 'the', 'second', 'one', '.', '[SEP]']
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
[('[CLS]', 0), ('this', 0), ('is', 0), ('the', 0), ('first', 0), ('sentence', 0), ('.', 0), ('[SEP]', 0), ('this', 1), ('is', 1), ('the', 1), ('second', 1), ('one', 1), ('.', 1), ('[SEP]', 1)]


We can use the tokenizer to preprocess the whole dataset. This works well, but it has the **disadvantage** of returning a dictionary (with our keys, input_ids, attention_mask, and token_type_ids, and values that are lists of lists). It will also only work if you have enough RAM to store your whole dataset during the tokenization

In [30]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)

In [31]:
print(tokenized_dataset.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


 We can use the **Dataset.map()** method which works by applying a function on each element of the dataset and also allows us some extra flexibility, if we need more preprocessing done than just tokenization.
 - **batched** - function is applied to multiple elements of our dataset at once
 - Datasets library applies this processing is by adding new fields to the datasets. Note that we could also have changed existing fields if our preprocessing function returned a new value for an existing key in the dataset to which we applied map().
 - Tokenizers library already uses **multiple threads** to tokenize our samples faster. We can also use multiprocessing when applying your preprocessing function with map() by passing along a **num_proc** argument.

In [32]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [33]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

### Dynamic Padding
**Collate function**- Responsible for putting together samples inside a batch, which can be passsed in **DataLoader**, the default being a function that will just convert your samples to PyTorch tensors and concatenate them.

**Dynamic Padding**- To apply padding as necessary on each batch and avoid having over-long inputs with a lot of padding. It will speed up training quite a bit but it can cause problems in **TPU**s- TPUs prefer fixed shapes, even when that requires extra padding.




In [34]:
from transformers import DataCollatorWithPadding
# Takes a tokenizer to know which padding token to use- left or on the right of the inputs
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [35]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

In [36]:
batch = data_collator(samples)
# 67 is the maximum length inside the batch
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'attention_mask': torch.Size([8, 67]),
 'labels': torch.Size([8])}

# Exercise
Replicate  the same with GLUE SST-2 dataset.

The **Stanford Sentiment Treebank** consists of sentences from movie reviews and human annotations of their sentiment. The task is to predict the sentiment of a given sentence. It uses the two-way (positive/negative) class split, with only sentence-level labels.

In [37]:
from datasets import load_dataset

sst2_datasets = load_dataset("glue", "sst2")
sst2_datasets

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [39]:
sst2_datasets["train"].features

{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [41]:
print(tokenizer(sst2_datasets["train"][0]["sentence"]))
print(tokenizer(sst2_datasets["train"][0]["sentence"], return_token_type_ids=False))


{'input_ids': [101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [47]:
def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True, return_token_type_ids=False)

In [45]:
tokenized_datasets = sst2_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [46]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})